In [1]:
import nltk
import pandas as pd
import io
import re
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from random import shuffle
#nltk.download('punkt')
#nltk.download('stopwords')
#nltk.download('PorterStemmer')

In [2]:
data = pd.read_csv('cleanerstill.csv', sep=";")
data.head()

/home/alanispani/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,age,body type,diet,drinks,drugs,education,essay0,essay4,essay6,essay7,...,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college university,about me i would love to think that i was so...,books absurdistan the republic of mice and ...,duality and humorous things,trying to find someone to hang out with i am ...,...,south san francisco california,doesnt have kids but might want them,straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef this is what that means 1 i am ...,i am die hard christopher moore fan i don't r...,,,...,oakland california,doesnt have kids but might want them,straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,english fluently spanish poorly french ...,single
2,38,thin,anything,socially,,graduated from masters program,i'm not ashamed of much but writing public te...,okay this is where the cultural matrix gets so...,,viewing listening dancing talking drinking...,...,san francisco california,,straight,has cats,,m,pisces but it doesnt matter,no,english french c,available
3,23,thin,vegetarian,socially,,working on college university,i work in a library and go to school,bataille celine beckett lynch jarmusch...,cats and german philosophy,,...,berkeley california,doesnt want kids,straight,likes cats,,m,pisces,no,english german poorly,single
4,29,athletic,,socially,never,graduated from college university,hey how's it going currently vague on the prof...,music bands rappers musicians at the moment...,,,...,san francisco california,,straight,likes dogs and likes cats,,m,aquarius,no,english,single


In [ ]:
essay4 = [essay for essay in data["essay4"]]
words = []
all_words = []
essay_list = []
essay_lister = []
stop_words = stopwords.words('english')
porter = PorterStemmer()

for i, essay in enumerate(essay4):
    tmp = []
    tmp_list = []
    if type(essay) != float:
        splt = essay.split()
        words.extend(splt)
        tmp.extend([porter.stem(w) for w in splt if not w in stop_words])
        for w in tmp:
            if "'" in w:
                tmp_list.extend(w.split("'"))
            else:
                tmp_list.append(w)
        essay_list.append((data["sex"][i],[word for word in tmp_list if word]))

for word in words:
    if "'" in word:
        all_words.extend([w for w in word.split("'") if not w in stop_words])
    else:
        all_words.append(word)

In [ ]:
clean_words = [w for w in all_words if not w in stop_words]
stemmed_words = [porter.stem(word) for word in clean_words]
freq_words = nltk.FreqDist(w for w in stemmed_words)
word_features = list(freq_words)[:2000]
print(word_features)

In [ ]:
def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

featuresets = [(document_features(t), f) for (f, t) in essay_list if t]
shuffle(featuresets)
train_set, test_set = featuresets[1000:], featuresets[:1000]

In [8]:
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.7


In [9]:
classifier.show_most_informative_features(100)

Most Informative Features
         contains(niven) = True                m : f      =      8.9 : 1.0
         contains(tiara) = True                f : m      =      7.0 : 1.0
       contains(haggard) = True                m : f      =      6.4 : 1.0
       contains(anathem) = True                m : f      =      6.1 : 1.0
           contains(sew) = True                f : m      =      5.9 : 1.0
           contains(err) = True                f : m      =      5.9 : 1.0
      contains(rothfuss) = True                m : f      =      5.8 : 1.0
       contains(anatomi) = True                f : m      =      5.6 : 1.0
        contains(lengua) = True                f : m      =      5.5 : 1.0
    contains(michaelson) = True                f : m      =      5.3 : 1.0
          contains(oryx) = True                f : m      =      5.2 : 1.0
        contains(ingrid) = True                f : m      =      5.0 : 1.0
          contains(wive) = True                f : m      =      4.8 : 1.0